In [1]:
!pip install --quiet transformers peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, GenerationConfig

In [4]:
model_id = "microsoft/phi-3.5-mini-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [5]:
print("Loading the base Phi-3.5-mini-instruct model in 4-bit...")

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    trust_remote_code=True,  # phi-3.5 sometimes requires this
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


Loading the base Phi-3.5-mini-instruct model in 4-bit...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [6]:
test_descriptions = [
    "Modern condo in downtown with rooftop pool and gym access",
    "Cozy treehouse nestled in the forest with hiking trails nearby",
    "Beachfront villa with private dock and sunset views",
    "Spacious city loft near trendy cafes and nightlife",
    "Rustic cabin surrounded by wildlife, perfect for digital detox"
]

In [7]:
def generate_zero_shot_title(model, description, max_new_tokens=20):
    """
    Generates a title using the base model (no fine-tuning).
    """
    # A simple zero-shot prompt. You can experiment with the wording.
    prompt = f"Write a short Airbnb listing title for this property:\n{description}\nTitle:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    generation_config = GenerationConfig(
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        do_sample=True,
    )

    with torch.no_grad():
        output_ids = model.generate(**inputs, **generation_config.__dict__)

    text_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Optional: parse out the portion after "Title:"
    if "Title:" in text_output:
        text_output = text_output.split("Title:")[-1].strip()

    return text_output

In [8]:
print("==== ZERO-SHOT TITLE GENERATION ====\n")

for i, desc in enumerate(test_descriptions, start=1):
    generated_title = generate_zero_shot_title(base_model, desc)
    print(f"Description {i}: {desc}")
    print(f"Zero-Shot Title {i}: {generated_title}\n")


Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


==== ZERO-SHOT TITLE GENERATION ====



Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Description 1: Modern condo in downtown with rooftop pool and gym access
Zero-Shot Title 1: "Sunrise Modern Condo - Downtown Pool & Gym Access, 4



Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Description 2: Cozy treehouse nestled in the forest with hiking trails nearby
Zero-Shot Title 2: "Enchanted Treehouse Retreat - Hiking Paradise Awaits!"



Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Description 3: Beachfront villa with private dock and sunset views
Zero-Shot Title 3: Exclusive Beachfront Villa with Private Dock & Sunset Views - Your Private Oasis



Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Description 4: Spacious city loft near trendy cafes and nightlife
Zero-Shot Title 4: "Chic City Loft - Urban Oasis, Cafe-Steeped Charm &

Description 5: Rustic cabin surrounded by wildlife, perfect for digital detox
Zero-Shot Title 5: 'Wild Nature Retreat: Your Next Escape to Nature's Embrace'



In [9]:
def generate_zero_shot_titles(model, description, max_new_tokens=20):
    """
    Generates titles using the base model (no fine-tuning) with different prompts.
    """
    # Define different prompt styles
    prompt_styles = [
        f"Write a short and catchy Airbnb title for this listing:\n{description}\nTitle:",
        f"Suggest an attractive title for an Airbnb property:\n{description}\nTitle:",
        f"Generate a unique and engaging title for an Airbnb rental:\n{description}\nTitle:",
        f"Create an inviting title for this Airbnb listing:\n{description}\nTitle:",
        f"Craft a compelling Airbnb listing title based on the following description:\n{description}\nTitle:",
    ]

    generation_config = GenerationConfig(
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        do_sample=True,
    )

    generated_titles = []
    for i, prompt in enumerate(prompt_styles, start=1):
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            output_ids = model.generate(**inputs, **generation_config.__dict__)
        text_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Extract only the generated title portion
        if "Title:" in text_output:
            text_output = text_output.split("Title:")[-1].strip()

        generated_titles.append((f"Prompt {i}", text_output))

    return generated_titles


# Run the experiment for all descriptions
print("==== ZERO-SHOT TITLE GENERATION WITH MULTIPLE PROMPTS ====\n")

for i, desc in enumerate(test_descriptions, start=1):
    print(f"\n=== Example {i} ===")
    print("Listing description:", desc)

    generated_titles = generate_zero_shot_titles(base_model, desc)
    for prompt_num, title in generated_titles:
        print(f"{prompt_num}: {title}")

    print("\n" + "="*50 + "\n")


Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


==== ZERO-SHOT TITLE GENERATION WITH MULTIPLE PROMPTS ====


=== Example 1 ===
Listing description: Modern condo in downtown with rooftop pool and gym access


Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `ma

Prompt 1: "Poolside Living: Luxury Condo with Gym & Pool in the Heart of the City
Prompt 2: "Chic Downtown Pool & Gym Companion: Modern Condo with Spl
Prompt 3: "Urban Oasis: Chic Modern Condo with Pool & Gym in Heart of
Prompt 4: "Luxurious Downtown Condo with Pool and Gym: Your Ideal Urban
Prompt 5: "Chic Downtown Condo with Pool & Gym: Modern Living at its Finest



=== Example 2 ===
Listing description: Cozy treehouse nestled in the forest with hiking trails nearby


Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `ma

...

Prompt 1: 🌴✨ Escape to Paradise - Private Beachfront Villa with Sun
Prompt 2: "Private Paradise: Beachfront Villa with Private Dock and Sunset Views"
Prompt 3: "Chic Beachfront Retreat with Private Dock - Sunset Views"
Prompt 4: Privileged Beach Haven: Beachfront Villa with Exclusive Dock & Sunset Pan
Prompt 5: Luxurious Beachfront Villa with Private Dock & Sunset Views - Your Perfect Es



=== Example 4 ===
Listing description: Spacious city loft near trendy cafes and nightlife


Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `ma

Prompt 1: "Urban Chic Loft: Your Ideal Neighborhood in the City"
Prompt 2: "Chic Urban Loft: Trendy Cafes & Lively Nightlife Just Ste
Prompt 3: "Urban Oasis: City Loft with Cafe Vibes and Nightlife at
Prompt 4: "Chic Urban Loft: Downtown Oasis with Cafe Culture & Nightlife"
Prompt 5: "Chic Loft in the Heart of the City: Coffee Connoisseur's



=== Example 5 ===
Listing description: Rustic cabin surrounded by wildlife, perfect for digital detox


Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=20) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Prompt 1: Experience Nature's Charm: Your Ideal Digital-Free Retreat
Prompt 2: "Tranquil Wild Retreat - Your Haven Away from the Digital World"
Prompt 3: "Wild Retreat: Embrace Nature in a Rustic Cabin for Digital Det
Prompt 4: "Wild Retreat: Embrace Nature's Call in Our Rustic Cabin
Prompt 5: "Wild Retreat: Embrace Nature & Disconnect in our Rustic Cabin


